## PytorchZeroToAll. Lec10

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(320, 10) # 320 -> 10

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1) # flatten the tensor
    x = self.fc(x)
    return F.log_softmax(x)

In [2]:
model = Net()
print(model)
total_params = sum(p.numel() for p in model.parameters())
print('\n',f'{total_params:,} total parameters.')

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

 8,490 total parameters.


## PytorchZeroToAll. Lec11

In [3]:
class InceptionA(nn.Module):
  def __init__(self, in_channels):
    super(InceptionA, self).__init__()
    self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

    self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

    self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
    self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

    self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

  def forwatd(self, x):
    branch1x1 = self.branch1x1(x)

    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)

    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

    branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch_pool = self.branch_pool(branch_pool)

    outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]

    return torch.cat(outputs, 1)

In [4]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

    self.incept1 = InceptionA(in_channels=10)
    self.incept2 = InceptionA(in_channels=20)

    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(1408, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = self.incept1(x)
    x = F.relu(self.mp(self.conv2(x)))
    x = self.incept2(x)
    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)

In [5]:
model = Net()
print(model)
total_params = sum(p.numel() for p in model.parameters())
print('\n',f'{total_params:,} total parameters.')

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(5, 5), stride=(1, 1))
  (incept1): InceptionA(
    (branch1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3dbl_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3dbl_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3dbl_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch_pool): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
  )
  (incept2): InceptionA(
    (branch1x1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3dbl_1): Conv2d(20, 16, kernel_size=(1, 1), stri